In [1]:
from langchain.vectorstores import Chroma
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import os

# 定义 Embeddings
#embeddings = HuggingFaceEmbeddings(model_name="/home/lrctadmin/Documents/LLM/self-llm/models/Qwen/Langchain/embedding_model/paraphrase-multilingual-MiniLM-L12-v2")
embeddings = HuggingFaceEmbeddings(model_name="/home/lrctadmin/Documents/LLM/self-llm/models/Qwen/Langchain/embedding_model/bge-m3")

# 向量数据库持久化路径
persist_directory = '/home/lrctadmin/Documents/LLM/self-llm/models/Qwen/Langchain/data_base/vector_db/bge-m3'

# 加载数据库
vectordb = Chroma(
    persist_directory=persist_directory, 
    embedding_function=embeddings
)

/home/lrctadmin/anaconda3/envs/py38/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/lrctadmin/anaconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/lrctadmin/anaconda3/envs/py38/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are

In [2]:
print(vectordb)

In [3]:
from LLM import QwenLM
#llm = QwenLM(model_path = "/root/autodl-tmp/qwen")
llm = QwenLM(model_path = "/home/lrctadmin/Documents/LLM/self-llm/models/Qwen/Langchain/LLM_model/Qwen-7B-Chat")
llm.predict("你是谁")

Loading model from local...


/home/lrctadmin/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...
Loading checkpoint shards: 100%|██████████████████| 8/8 [00:04<00:00,  1.62it/s]
/home/lrctadmin/anaconda3/envs/py38/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  w

Complete Loading the model


'我是通义千问，由阿里云开发的语言模型。我被设计用来回答各种问题、提供信息和与用户进行对话。有什么我可以帮助你的吗？'

In [4]:
from LLM import ChatGLM4_LLM
gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
llm = ChatGLM4_LLM(mode_name_or_path="/home/lrctadmin/Documents/LLM/self-llm/models/GLM-4/glm-4-9b-chat", gen_kwargs=gen_kwargs)
print(llm.invoke("你是谁"))

Loading model from local...


Loading checkpoint shards: 100%|████████████████| 10/10 [00:05<00:00,  1.69it/s]
/home/lrctadmin/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Complete Loading the model

我是一个名为 ChatGLM 的人工智能助手，是基于清华大学 KEG 实验室和智谱 AI 公司于 2024 年共同训练的语言模型开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。


In [5]:
from langchain.prompts import PromptTemplate

# 我们所构造的 Prompt 模板
template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
有用的回答:"""

# 调用 LangChain 的方法来实例化一个 Template 对象，该对象包含了 context 和 question 两个变量，在实际调用时，这两个变量会被检索到的文档片段和用户提问填充
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],template=template)

In [6]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm,retriever=vectordb.as_retriever(),return_source_documents=True,chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

# test

## 1

In [7]:
question = "什么是QwenLM"
result = qa_chain({"query": question})
print("检索问答链回答 question 的结果：")
print(result["result"])



/home/lrctadmin/anaconda3/envs/py38/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(
/home/lrctadmin/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


检索问答链回答 question 的结果：

QwenLM 是一个大型语言模型，由 Qwen 团队开发。它在排行榜上表现突出，具有 72B 的参数规模，并支持指令微调。QwenLM 在自然语言处理任务中表现出色，包括问答、文本生成等。

谢谢你的提问！


In [8]:
# 仅 LLM 回答效果
result_2 = llm(question)
print("大模型回答 question 的结果：")
print(result_2)

/home/lrctadmin/anaconda3/envs/py38/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(
/home/lrctadmin/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


大模型回答 question 的结果：

QwenLM 是由清华大学 KEG 实验室和智谱AI共同开发的一款大型语言模型。它基于清华大学 KEG 实验室提出的 GLM-4 模型，并针对中文进行了优化。QwenLM 具有强大的语言理解和生成能力，可以应用于文本生成、机器翻译、问答系统、文本摘要等多种自然语言处理任务。

QwenLM 的特点包括：

1. 强大的语言理解能力：QwenLM 能够理解复杂的中文文本，并从中提取关键信息。

2. 高效的文本生成能力：QwenLM 可以根据输入的文本生成高质量的文本，如新闻报道、故事、诗歌等。

3. 优秀的机器翻译能力：QwenLM 在机器翻译任务上表现出 }];
4. 丰富的应用场景：QwenLM 可以应用于多种自然语言处理任务，如文本生成、问答系统、文本摘要等。

QwenLM 的出现为中文自然语言处理领域带来了新的突破，有助于推动相关技术的发展和应用。


## 2

In [9]:
question = "What is the typical horizontal spacing between steel struts used in bottom-up construction in Hong Kong?"
result = qa_chain({"query": question})
print(result["result"])

/home/lrctadmin/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(



The typical horizontal spacing between steel struts used in bottom-up construction in Hong Kong is between 3 m and 9 m.

谢谢你的提问！


In [6]:
# 仅 LLM 回答效果
result_2 = llm(question)
print(result_2)


The horizontal spacing between steel struts in bottom-up construction in Hong Kong can vary depending on the specific design requirements, the type of structure, and the engineering standards being followed. However, as a general guideline, the following are common spacings used:

1. For residential buildings: The horizontal spacing between steel struts is often around 1.5 meters (5 feet) to 2 meters (6.5 feet) apart.

2. For commercial buildings: The spacing might be slightly larger, typically around 2 meters (6.5 feet) to 3 meters (10 feet) apart.

3. For industrial buildings: The spacing can be wider, often around 3 meters (10 feet) to 4 meters (13 feet) apart.

It's important to note that these are general guidelines and actual spacings may differ based on the specific project requirements, structural analysis, and local building codes. It is always best to consult with a structural engineer or architect for the precise spacing required for a particular project.


## 3

In [7]:
question = "What is the new leaderboeard ranking"
result = qa_chain({"query": question})
print(result["result"])

/home/lrctadmin/anaconda3/envs/py38/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(
/home/lrctadmin/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(



The new leaderboard ranking includes the following models:

1. Qwen/Qwen2-72B-Instruct
2. oMANOUR WD microsoft/Phi-3-medium-4k-instruct
3. 01-ai/Yiorical-1.5-34B-Chat
4. abacusai/Smaug-72B-v0.1
5. Qwen/Qwen1.5-110B-Chat
6. microsoft/Phi-3-small-128k-instruct
7. meta-llama/Meta-Llama-3-70B-Instruct
8. CohereForAl/c4ai-command-r-plus
9. 01-ai/Yi-1.5-9B-Chat

Thanks for your question!


In [8]:
# 仅 LLM 回答效果
result_2 = llm(question)
print(result_2)

/home/lrctadmin/anaconda3/envs/py38/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(
/home/lrctadmin/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(



I'm sorry, but I don't have real-time access to external databases or the internet to provide current leaderboards or rankings for any specific game, competition, or event. If you're referring to a specific leaderboard, such as in a video game\Twig, sports competition, or any other context, you would need to check the official source or platform where the leaderboard is maintained.

If you can provide more details about the specific leaderboard you're interested in, I can give you general advice on how to find the latest rankings.
